In [11]:
import os,sys
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.0 pyspark-shell'

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

Aşağıdaki kod çalışıyor

In [ ]:
from __future__ import print_function

import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import *

if __name__ == "__main__":

    spark = SparkSession\
        .builder\
        .appName("StructuredKafkaWordCount")\
        .getOrCreate()

    # Create DataSet representing the stream of input lines from kafka
    lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "Stats")\
        .load()\
        .selectExpr("CAST(value AS STRING)")


    # Split the lines into words
    words = lines\
        .select(
        # explode turns each item in an array into a separate row
        explode(
            split(lines.value, ' ')
        ).alias('word') 
    )

    words.printSchema()

    # Generate running word count
    wordCounts = words\
        .groupBy('word').count()

    wordCounts = wordCounts.withColumn("jsonColumn", to_json(struct([wordCounts[x] for x in wordCounts.columns])))

    wordCounts.printSchema()

    # Start running the query that prints the running counts to the console
query = wordCounts\
        .selectExpr("CAST(jsonColumn AS string) as value")\
        .writeStream\
        .format('kafka')\
        .option('kafka.bootstrap.servers', 'localhost:9092')\
        .option('topic', 'Action')\
        .option('checkpointLocation', '/home/ubuntu/spark/chkpoint/')\
        .outputMode('complete')\
        .start()

query.awaitTermination()

Aşağıdaki kod append modda hata vermekte, complete yapılırsa düzeliyor

In [ ]:
from __future__ import print_function

import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import *

if __name__ == "__main__":

    spark = SparkSession\
        .builder\
        .appName("StructuredKafkaWordCount")\
        .getOrCreate()

    # Create DataSet representing the stream of input lines from kafka
    lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "Stats")\
        .load()\
        .selectExpr("CAST(value AS STRING)","CAST(timestamp AS timestamp)")

    lines.printSchema()

    # Split the lines into words
    words = lines\
        .withWatermark("timestamp","10 seconds")\
        .select(
        # explode turns each item in an array into a separate row
        explode(
            split(lines.value, ' ')
        ).alias('word') , "timestamp"
    )

    words.printSchema()

    # Generate running word count
    wordCounts = words\
        .withWatermark("timestamp","10 seconds")\
        .groupBy('word').count()

    wordCounts = wordCounts.withColumn("jsonColumn", to_json(struct([wordCounts[x] for x in wordCounts.columns])))

    wordCounts.printSchema()

    # Start running the query that prints the running counts to the console
query = wordCounts\
        .selectExpr("CAST(jsonColumn AS string) as value")\
        .writeStream\
        .format('kafka')\
        .option('kafka.bootstrap.servers', 'localhost:9092')\
        .option('topic', 'Action')\
        .option('checkpointLocation', '/home/ubuntu/spark/chkpoint/')\
        .outputMode('append')\
        .start()

query.awaitTermination()

## Jupyter'de Bash Kullanımı

In [ ]:
%%bash 
/home/ubuntu/kafka_2.12-1.1.0/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic test
/home/ubuntu/kafka_2.12-1.1.0/bin/kafka-topics.sh --list --zookeeper localhost:2181

### İncelenebilecek Kaynaklar:

https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-apache-kafka-spark-structured-streaming
https://docs.microsoft.com/en-us/azure/hdinsight/spark/apache-spark-jupyter-notebook-use-external-packages